In [1]:
from climso.detection import Prominences

import os, re
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import pandas as pd
import numpy as np


In [2]:
base_dir='Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/'
files = [base_dir+file for file in os.listdir(base_dir) if file.startswith('imoa_06563_c1_')]

def extract_timestamp(file_path):
    match = re.search(r'\d{8}_\d{8}', file_path)
    if match:
        return match.group(0)
    return ''

files = sorted(files, key=extract_timestamp)
    
files

['Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06055568_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06074474_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06111447_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06123511_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06152664_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06162683_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06172657_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06222710_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06232765_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/2024-07-20/imoa_06563_c1_20240720_06242568_bri1.fts.bz2',
 'Y:/data/CLIMSO/2024/data_calibrated/20

In [3]:
file = files[-10]

In [10]:
prominences = Prominences(file)

dtype('float64')

In [ ]:
deg = prom_degs[0]

max_radiuses = []
degs    = []
times   = []
speeds  = []
fileset = []

for n in range(1, len(files)):
    array1, date_obs1 = radialArray(files[n-1])
    array2, date_obs2 = radialArray(files[n])

    max_radius_entry_1 = maxRadius(array1, deg)
    if max_radius_entry_1 is None: continue
    max_radius_entry_2 = maxRadius(array2, max_radius_entry_1['deg'])
    if max_radius_entry_2 is None: continue
    
    deg = int(max_radius_entry_2['deg'])

    distance = (max_radius_entry_2['radius'] - max_radius_entry_1['radius'])*696e3 # km
    time = abs(date_obs2 - date_obs1)
    if time.seconds == 0 : continue # to avoid division by zero
    
    speed = distance / time.seconds # km/s
    
    max_radiuses.append(max_radius_entry_1['radius'])
    degs.append(max_radius_entry_1['deg'])
    times.append(date_obs1)
    speeds.append(speed)
    fileset.append(files[n-1])

if max_radius_entry_2:
    max_radiuses.append(max_radius_entry_2['radius'])
    degs.append(max_radius_entry_2['deg'])
    times.append(date_obs2)
    fileset.append(files[-1])